# DataOverflow Ramathon

https://www.kaggle.com/competitions/dataoverflow-ramathon/overview/description

In [1]:
#!pip install -U dataprep

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#from dataprep.eda import create_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
import warnings
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
# machine learning
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

warnings.filterwarnings('ignore')

### Reading Data

In [4]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
sample=pd.read_csv('sample_submission.csv')

In [5]:
train.dropna(inplace=True)
train = train.drop('IMPORTER.TIN', 1)
test = test.drop('IMPORTER.TIN', 1)

In [6]:
scale= StandardScaler()
scaled_data = scale.fit_transform(train[['CIF_USD_EQUIVALENT', 'QUANTITY', 'GROSS.WEIGHT', 'TOTAL.TAXES.USD']]) 
train[['CIF_USD_EQUIVALENT', 'QUANTITY', 'GROSS.WEIGHT', 'TOTAL.TAXES.USD']] = scaled_data

In [7]:
# Handling categorical attributes

ordinal_encoder = OrdinalEncoder()
train['OFFICE'] = ordinal_encoder.fit_transform(train[['OFFICE']])
#train['IMPORTER.TIN'] = ordinal_encoder.fit_transform(train[['IMPORTER.TIN']])
train['ORIGIN.CODE'] = ordinal_encoder.fit_transform(train[['ORIGIN.CODE']])


test['OFFICE'] = ordinal_encoder.fit_transform(test[['OFFICE']])
#test['IMPORTER.TIN'] = ordinal_encoder.fit_transform(test[['IMPORTER.TIN']])
test['ORIGIN.CODE'] = ordinal_encoder.fit_transform(test[['ORIGIN.CODE']])
'''
transformer = make_column_transformer(
    (MinMaxScaler(), 
        ['CIF_USD_EQUIVALENT','QUANTITY', 'GROSS.WEIGHT','TOTAL.TAXES.USD','TARIFF.CODE']),
    (OneHotEncoder(handle_unknown='ignore'), 
        ['OFFICE','ORIGIN.CODE']))

# Fit
transformer.fit(train)

# Apply the transformation
train = transformer.transform(train)
test = transformer.transform(test)
'''

"\ntransformer = make_column_transformer(\n    (MinMaxScaler(), \n        ['CIF_USD_EQUIVALENT','QUANTITY', 'GROSS.WEIGHT','TOTAL.TAXES.USD','TARIFF.CODE']),\n    (OneHotEncoder(handle_unknown='ignore'), \n        ['OFFICE','ORIGIN.CODE']))\n\n# Fit\ntransformer.fit(train)\n\n# Apply the transformation\ntrain = transformer.transform(train)\ntest = transformer.transform(test)\n"

In [8]:
train.head()

,ID,OFFICE,TARIFF.CODE,ORIGIN.CODE,CIF_USD_EQUIVALENT,QUANTITY,GROSS.WEIGHT,TOTAL.TAXES.USD,illicit,date
0,0,62.0,8703232225,83.0,0.395796,-0.097863,-0.133495,-0.029640,0,2013-01-01
1,1,41.0,8703321220,40.0,-0.183059,-0.097863,0.170308,-0.220362,0,2013-01-01
2,2,61.0,8704212025,83.0,-0.199753,-0.097863,0.685012,0.111631,0,2013-01-01
3,3,62.0,3901100000,24.0,-0.183192,-0.059830,-0.137239,-0.049420,0,2013-01-01
4,4,61.0,8517180000,94.0,-0.199395,0.034942,-0.184665,-0.215002,0,2013-01-01


In [9]:
print("Shape of Train Dataset: "+str(train.shape))
print("Shape of Test Dataset: "+str(test.shape))

Shape of Train Dataset: (157244, 10)
Shape of Test Dataset: (40000, 9)


#### If we exclude the ID column so we have 9 columns to use as features and we are going to predict the illicit column

### Let's check the type of these columns

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157244 entries, 0 to 159999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID                  157244 non-null  int64  
 1   OFFICE              157244 non-null  float64
 2   TARIFF.CODE         157244 non-null  int64  
 3   ORIGIN.CODE         157244 non-null  float64
 4   CIF_USD_EQUIVALENT  157244 non-null  float64
 5   QUANTITY            157244 non-null  float64
 6   GROSS.WEIGHT        157244 non-null  float64
 7   TOTAL.TAXES.USD     157244 non-null  float64
 8   illicit             157244 non-null  int64  
 9   date                157244 non-null  object 
dtypes: float64(6), int64(3), object(1)
memory usage: 13.2+ MB


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  40000 non-null  int64  
 1   OFFICE              40000 non-null  float64
 2   TARIFF.CODE         40000 non-null  int64  
 3   ORIGIN.CODE         40000 non-null  float64
 4   CIF_USD_EQUIVALENT  40000 non-null  float64
 5   QUANTITY            40000 non-null  float64
 6   GROSS.WEIGHT        40000 non-null  float64
 7   TOTAL.TAXES.USD     40000 non-null  float64
 8   date                39786 non-null  object 
dtypes: float64(6), int64(2), object(1)
memory usage: 2.7+ MB


#### We have 4 columns considered as Categorical features and the rest are numerical features

#### Let's check for missing values

In [12]:
train.isna().sum()

ID                    0
OFFICE                0
TARIFF.CODE           0
ORIGIN.CODE           0
CIF_USD_EQUIVALENT    0
QUANTITY              0
GROSS.WEIGHT          0
TOTAL.TAXES.USD       0
illicit               0
date                  0
dtype: int64

In [13]:
test.isna().sum()

ID                      0
OFFICE                  0
TARIFF.CODE             0
ORIGIN.CODE             0
CIF_USD_EQUIVALENT      0
QUANTITY                0
GROSS.WEIGHT            0
TOTAL.TAXES.USD         0
date                  214
dtype: int64

### To have quick EDA we will use a powerful library that provides insights on the dataset

In [14]:
#report = create_report(train, title='My Report')

In [15]:
#report

### The target variable is very unbalanced so you need to figure out how to fight overfitting in this problem 

### What about Categorical variable ? How you are going to deal with them ? One Hot encoding ? Label Encoding ? or drop some ? 

#### For now we will use only the numerical features

In [16]:
train.columns

Index(['ID', 'OFFICE', 'TARIFF.CODE', 'ORIGIN.CODE', 'CIF_USD_EQUIVALENT',
       'QUANTITY', 'GROSS.WEIGHT', 'TOTAL.TAXES.USD', 'illicit', 'date'],
      dtype='object')

In [17]:
main_cols=['CIF_USD_EQUIVALENT', 'QUANTITY', 'GROSS.WEIGHT', 'TOTAL.TAXES.USD', 'OFFICE']#, 'ORIGIN.CODE'] #test

In [18]:
X=train[main_cols]
y=train.illicit

In [19]:
'''
%pip install imbalanced-learn
# transform the dataset
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
from collections import Counter
counter = Counter(y)
print(counter)
'''

'\n%pip install imbalanced-learn\n# transform the dataset\nfrom imblearn.over_sampling import SMOTE\noversample = SMOTE()\nX, y = oversample.fit_resample(X, y)\n# summarize the new class distribution\nfrom collections import Counter\ncounter = Counter(y)\nprint(counter)\n'

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42, stratify=y)

In [21]:
# Support Vector Machines


In [22]:
Ridge= RidgeClassifierCV()
grad_boost = GradientBoostingClassifier(n_estimators = 100)
random_forest = RandomForestClassifier(n_estimators=100)
decision_tree = DecisionTreeClassifier()
perceptron = Perceptron()
gaussian = GaussianNB()
svc = SVC()
lr = LogisticRegression()
knn = KNN()
dt = DecisionTreeClassifier()

classifiers = [
    ('Gradient Boosting', grad_boost),
    ('Random Forest', random_forest),
               ('Decision Tree', decision_tree),
               ('K Nearest Neighbours', knn),
               ('Classification Tree', dt)]

#clf.fit(X_train,y_train)

In [23]:
for clf_name, clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print('{:s} : {:.3f}'.format(clf_name, accuracy_score(y_test, y_pred)))

Gradient Boosting : 0.930
Random Forest : 0.929
Decision Tree : 0.882
K Nearest Neighbours : 0.923
Classification Tree : 0.882


In [24]:
vc = VotingClassifier(estimators=classifiers)

vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.9283690813435856


In [25]:
'''
y_pred=clf.predict(X_test)
#print(roc_auc_score(y_pred,y_test))
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))
'''

'\ny_pred=clf.predict(X_test)\n#print(roc_auc_score(y_pred,y_test))\nprint(clf.score(X_train, y_train))\nprint(clf.score(X_test, y_test))\n'

In [26]:
sample['illicit']=vc.predict(test[main_cols])

In [27]:
sample.illicit.value_counts()

0    39984
1       16
Name: illicit, dtype: int64

In [28]:
sample.head()

,ID,illicit
0,160000,0
1,160001,0
2,160002,0
3,160003,0
4,160004,0


In [29]:
sample.to_csv("Baseline.csv",index=False)